In [1]:
# Third party packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import display
import plotly.express as px

# Adding path
from os.path import abspath
import sys
sys.path.insert(0, abspath('..'))

# 
from query_request import *
from query_def import *
import credentials
# from sqlalchemy import create_engine

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth=None


In [2]:
# Demo for LDM

# Upload dataset
# downlaod dataset

In [3]:
# Demo for dragoman

# [Creating Endpoint](../configs/docker_command_icews.sh)

In [4]:
# create rdfmts
from DeTrusty import run_query
from DeTrusty.Molecule.MTCreation import create_rdfmts
from DeTrusty.Molecule.MTManager import ConfigFile

# Creating endpoints dictionary
endpoints = dictfrmjson('../configs/endpoints_sample.json')
print(endpoints)

# Creating RDF molecule template (rdfmts) from endpoints
# log_rdfmts = create_rdfmts(endpoints, '../configs/rdfmts.json')

# if already rdfmts has been created 
config_private = ConfigFile('../configs/rdfmts_private.json')
config_public = ConfigFile('../configs/rdfmts_public.json')

Total time taken by dictfrmjson function: 0.0003600120544433594
{'https://labs.tib.eu/sdm/worldbank_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/icews_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/lei_endpoint/sparql': {}, 'https://skynet.coypu.org/coypu-internal': {'username': 'Your UserName', 'password': 'Your Password'}}


## Endpoints Status

In [5]:
# Demo query to all available enpoints

# demo query
query_test = """SELECT * WHERE{ ?Subject a ?Concept }LIMIT 1000"""


# creating request for cmemc
display("############### Cmemc Endpoint ###############")
cmemc_request = SPARQLRequest(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
# executing query and printing response
cmemc_request.execute(query_test)
buf = StringIO(cmemc_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for skynet
display("############### Skynet Endpoint ###############")
skynet_request = SPARQLRequest(skynet_endpoint, skynet_user, skynet_pass, 'basic')
# executing query and printing response
skynet_request.execute(query_test)
buf = StringIO(skynet_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for worldbank
display("############### Worldbank Endpoint ###############")
worldbank_request = SPARQLRequest(worldbank_endpoint)
# executing query and printing response
worldbank_request.execute(query_test)
display(worldbank_request.response.status_code)
buf = StringIO(worldbank_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for LEI
display("############### LEI Endpoint ###############")
lei_request = SPARQLRequest(lei_endpoint)
# executing query and printing response
lei_request.execute(query_test)
buf = StringIO(lei_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for ICEWS
display("############### ICEWS Endpoint ###############")
icews_request = SPARQLRequest(icews_endpoint)
# executing query and printing response
icews_request.execute(query_test)
buf = StringIO(icews_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))


'############### Cmemc Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.417449951171875


,Subject,Concept
995,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/2000/01/rdf-schema#Resource
996,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/ns/shacl#PropertyShape
997,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
998,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2000/01/rdf-schema#Resource
999,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2002/07/owl#DatatypeProperty


'############### Skynet Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.10377740859985352


...

Passed Query Status: 200
Total time taken by execute function: 0.0649559497833252


,Subject,Concept
995,https://data.coypu.org/lei/2138006HHBEMF1TGWU29-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
996,https://data.coypu.org/lei/2138006HHUG8LC4UD387-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
997,https://data.coypu.org/lei/2138006HHXWLEK5WVK02-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
998,https://data.coypu.org/lei/2138006HI1P4JTM2WM22-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
999,https://data.coypu.org/lei/2138006HI5AF5E1P4T95-LA,https://www.gleif.org/ontology/Base/PhysicalAddress


'############### ICEWS Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.06700277328491211


,Subject,Concept
995,https://data.coypu.org/icews/36616020,https://schema.coypu.org/global#Conflict
996,https://data.coypu.org/icews/36616021,https://schema.coypu.org/global#Conflict
997,https://data.coypu.org/icews/36616024,https://schema.coypu.org/global#Conflict
998,https://data.coypu.org/icews/36616025,https://schema.coypu.org/global#Conflict
999,https://data.coypu.org/icews/36616026,https://schema.coypu.org/global#Conflict


## Federated Query

In [6]:
# creating request for Federated Query
display("############### Federated Query ###############")
fdq_request = SPARQLRequest(is_fdq=True)
fdq_request.execute(query_test, config_public, False)
fdq_request.save('../query_results/', 'query_test')

'############### Federated Query ###############'

2022-12-01 15:45:09,208 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/icews_endpoint/sparql/
2022-12-01 15:45:09,217 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql/
2022-12-01 15:45:09,221 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/lei_endpoint/sparql/


Total time taken by execute function: 0.6942243576049805
Index                    128
Subject.value           8000
Subject.type            8000
Concept.value           8000
Concept.type            8000
__meta__.is_verified    1000
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Subject.value         1000 non-null   object
 1   Subject.type          1000 non-null   object
 2   Concept.value         1000 non-null   object
 3   Concept.type          1000 non-null   object
 4   __meta__.is_verified  1000 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 32.4+ KB
None
Total time taken by json_to_csv function: 0.04970550537109375
Total time taken by save function: 0.05005836486816406


In [7]:
# print(query_2_fdq_ex_1)

query = """PREFIX wb: <http://worldbank.org/>
PREFIX wbi: <http://worldbank.org/Indicator/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?year (AVG(?year_exp_WB) as ?year_avg_exp_WB) 
WHERE {
    ?country a wb:Country .
    ?country dc:identifier "DEU" .
    ?country rdfs:label ?country_name.
    ?country owl:sameAs ?sameAsCountry .
    ?country wb:hasAnnualIndicatorEntry ?annualIndicator .
    
    ?annualIndicator wb:hasIndicator <http://worldbank.org/Indicator/SP.DYN.LE00.IN> .
    ?annualIndicator owl:hasValue ?year_exp_WB .
    ?annualIndicator wb:worldBankDateYear ?year .
}
GROUP BY ?year
"""

fdq_request.execute(query, config_private, False)
fdq_request.save('../query_results/', 'query')

2022-12-01 15:45:09,938 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal
2022-12-01 15:45:09,942 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql
2022-12-01 15:45:09,948 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql
2022-12-01 15:45:09,934 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal


Total time taken by execute function: 17.360366344451904
Index                    128
year.value               488
year.type                488
year_avg_exp_WB.value    488
year_avg_exp_WB.type     488
__meta__.is_verified      61
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year.value             61 non-null     object
 1   year.type              61 non-null     object
 2   year_avg_exp_WB.value  61 non-null     object
 3   year_avg_exp_WB.type   61 non-null     object
 4   __meta__.is_verified   61 non-null     bool  
dtypes: bool(1), object(4)
memory usage: 2.1+ KB
None
Total time taken by json_to_csv function: 0.03937339782714844
Total time taken by save function: 0.039598703384399414


In [8]:
df = pd.read_csv('../../query_results/query_2_fdq_ex_1.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
df['WikiData-WB(AvgLifeExpectancy)'] = np.abs(df['year_avg_exp_wiki']-df['year_avg_exp_WB'])
df.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../../query_results/query_2_fdq_ex_1.csv'

In [ ]:
fig = px.choropleth(df[df['year_WB']==2016], locations="country_code",
                    color="WikiData-WB(AvgLifeExpectancy)", # lifeExp is a column of gapminder
                    hover_name="country_name", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Magenta)
fig.write_html('../../query_results/query_2_fdq_ex_1.html')
fig.show()